  #### set up telegram notifications

  не очень понятно, нужно ли это.

  если нужно -- напишите @oserikov в телеграме, я расскажу, что сделать,
  чтобы присылались сообщения с качеством модели когда она отработает.

In [0]:
telegram_notifications_enabled=False
EXP_DESCRIPTION = "BASELINE"

if telegram_notifications_enabled:
    bot_token = input("введите telegram bot token: ")
    chat_id = "292749902" # for @oserikov


 #### install prereqs

In [0]:
get_ipython().system(f"git clone https://github.com/NIS-2018-CROSS-M/colab-tools.git")
get_ipython().magic(f"cd colab-tools")
get_ipython().system(f"bash colab-install-opennmt.sh")
get_ipython().system(f"bash colab-install-cuda92-pytorch41.sh")
get_ipython().system(f"bash colab-install-torchtext.sh")
get_ipython().magic(f"cd ..")



Cloning into 'colab-tools'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 14 (delta 3), reused 13 (delta 2), pack-reused 0
Unpacking objects: 100% (14/14), done.
/content/colab-tools
Cloning into 'OpenNMT-py'...
remote: Enumerating objects: 13531, done.
remote: Total 13531 (delta 0), reused 0 (delta 0), pack-reused 13531
Receiving objects: 100% (13531/13531), 145.37 MiB | 29.63 MiB/s, done.
Resolving deltas: 100% (9640/9640), done.
Switched to a new branch 'stable-version'
--2019-03-15 16:49:41--  https://developer.nvidia.com/compute/cuda/9.2/Prod2/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.148-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.162.216
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.162.216|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/

In [0]:
# install dependencies used in calma project
get_ipython().system('/usr/bin/python3 -m pip install configargparse')
get_ipython().system('git clone https://github.com/NIS-2018-CROSS-M/calma_tools.git')

# receive the calma
get_ipython().system('git clone https://github.com/NIS-2018-CROSS-M/vardial-shared-task')
get_ipython().magic('cd vardial-shared-task')
get_ipython().system('rm onmt-data/*')
get_ipython().system('rm results/*')
get_ipython().magic('cd ..')



In [0]:
from collections import defaultdict as dd
from random import shuffle
import re
import sys
sys.path.append(get_ipython().getoutput("readlink -e calma_tools")[0])
from calma_tools.ml_util import MLUtil
import urllib



In [0]:
get_ipython().magic('cd vardial-shared-task')



In [0]:
train_steps=10000
valid_steps=1000
save_checkpoint_steps = valid_steps

train_params = [
    f"-train_steps {train_steps}",
    f"-valid_steps {valid_steps}",
    f"-save_checkpoint_steps {save_checkpoint_steps}",
    f"-world_size 1",
    f"-gpu_ranks 0 1",
    f"-encoder_type brnn"
]

pred_params = [
    f"-replace_unk",
    f"-verbose",
    f"-n_best 8",
    f"-beam 8"
]


  #### data modification

In [0]:
class TrainDataModifyer:
    @staticmethod
    def modify_src_line(line):
        return line


    @staticmethod
    def restore_src_line(line):
        return line


    @staticmethod
    def modify_tgt_line(line):
        return line


    @staticmethod
    def restore_tgt_line(line):
        return line


class NBestDataModifyer:
    @staticmethod
    def sent_to_baseline_compatible(line):
        return line
                       
    @staticmethod
    def hyp_to_baseline_compatible(line):
        return line

    
class DataEvaluator:
    otypes = ["analysis","lemma","tag"]
    
    @staticmethod
    def update_data(data, line):
        lan, wf, lemma, pos, msd = line.split('\t')
        
        data["analysis"][wf].add((lemma,pos,msd))
        data["lemma"][wf].add(lemma)
        data["tag"][wf].add((pos,msd))
        
        return data


  #### ml

In [0]:
# filenames, many of them
train_uncovered_filename = f"train/trk-uncovered-transliterated"
train_res_src_filename = f"onmt-data/trk-src-train.txt"
train_res_tgt_filename = f"onmt-data/trk-tgt-train.txt"

test_covered_filename = f"test/trk-covered-transliterated"


test_res_src_filename = f"onmt-data/trk-src-test.txt"
test_res_tgt_filename = f"onmt-data/trk-tgt-test.txt"
test_pred_output_filename = f"results/trk-test-covered.sys" # output :)

val_covered_filename = f"dev/trk-covered-transliterated"
val_uncovered_filename = f"dev/trk-uncovered-transliterated"
val_res_src_filename = f"onmt-data/trk-src-dev.txt"
val_res_tgt_filename = f"onmt-data/trk-tgt-dev.txt"
val_pred_output_filename = f"results/trk-dev-covered.sys" # output :)


model_filename = f"models/trk.model"

score_log_filename = f"trk-score.log"

def ml(train_params, prediction_params, dataModifyer, nbestModifyer, dataEvaluator):
    mlUtil = MLUtil(prediction_params, dataModifyer, nbestModifyer)

    get_ipython().system(f'touch {score_log_filename}')


    # ml| data preprocessing
    mlUtil.generate_data(train_uncovered_filename, train_res_src_filename, train_res_tgt_filename)
    mlUtil.generate_data(val_uncovered_filename, val_res_src_filename, val_res_tgt_filename)
    mlUtil.generate_data(test_covered_filename, test_res_src_filename, test_res_tgt_filename)

    # ml| training
    mlUtil.train(train_res_src_filename, train_res_tgt_filename, val_res_src_filename, val_res_tgt_filename, model_filename, train_params)


    mlUtil.predict(model_filename, test_res_src_filename, test_covered_filename, test_pred_output_filename)


    mlUtil.predict(model_filename, val_res_src_filename, val_covered_filename, val_pred_output_filename)

    get_ipython().system(f'echo "*===QUALITY ON VAL DATA===*" >> {score_log_filename}')
    mlUtil.score_predictions(val_pred_output_filename, val_uncovered_filename, score_log_filename, dataEvaluator)

    # log eval results
    get_ipython().system(f'cat {score_log_filename}')

    # send eval to @oserikov at telegram
    if telegram_notifications_enabled:
        telegram_message = f"#score\n{lang}\n{track}\n"+''.join(open(score_log_filename).readlines())+'\n'+EXP_DESCRIPTION

        telegram_message_encoded = urllib.parse.quote(telegram_message)
        get_ipython().system(f'curl -i -X GET "https://api.telegram.org/bot{bot_token}/sendMessage?chat_id={chat_id}&text={telegram_message_encoded}&parse_mode=markdown"')



In [0]:
!head {val_pred_output_filename}

In [0]:
ml(train_params, pred_params, TrainDataModifyer, NBestDataModifyer, DataEvaluator)


  # sandbox

In [0]:
# !!! mlUtil.score_predictions(val_pred_output_filename, val_uncovered_filename, score_log_filename, dataEvaluator)

In [0]:
from google.colab import files
files.download(val_pred_output_filename)